In [1]:
import pandas as pd
f = open("c:/data/text/news1.txt")
lines = f.readlines()
f.close()

In [2]:
import re
from konlpy.tag import Hannanum

han = Hannanum()
word_list = []
for i in range(len(lines)):
    #한글, 영어 외에는 공백
    sentence =re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', lines[i])
    #명사만 추출
    a=sentence.strip()
    n=han.nouns(a)
    #한글자짜리 제외
    n2=[x for x in n if len(x)>1]
    if len(n2)>0:
        word_list.append(n2)

# tf-idf 행렬을 만들기 위해 다시 역토큰화
detokenized_doc = []
for i in range(len(word_list)):
    t = ' '.join(word_list[i])
    detokenized_doc.append(t)
detokenized_doc

['츄파춥스 새콤달콤한 사탕 오랫동안 세계인 사랑 브랜드 스페인 출시 지금 여개국 가지 종류 판매 올해 주년 글로벌 브랜드 츄파춥스 우리나라 중소기업 핸드허그 라이선스 계약 소식',
 '이달 츄파춥스 로고 사탕 그림 생활용품 문구류 여개 상품 이달 시중 판매 예정 라이선스 계약 상표 브랜드 이미지 사용 특정 지역 제품 범위 한해 제조 유통 허가 권리 획득 인지도 저명성 츄파춥스 지식재산 활용 것인데요 핸드허그 제품 판매금액 일정 부분 츄파춥스 방식 계약 기간 내년 츄파춥스 그동안 각종 의류 식료품 회사 콜라보 한국에서 다양 상품군 출시 처음',
 '설립 핸드허그 크리에이터 커머스 플랫폼 캐릭터 제작 여개 크리에이터 편의점 상품 각종 굿즈 판매 이름 일례 꽃카 캐릭터 편의점 출시 사랑 핸드허그 츄파춥스 글로벌 브랜드 협업 라이선스 브랜드 론칭하 처음 그동안 확보 시장 영향력 전문성 활용 계획 츄파춥스와 계약 핸드허그 제품 생산 유통 디자인 마케팅 물류 인프라까지 원스톱 시스템 때문 가능']

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(detokenized_doc)
X.shape # TF-IDF 행렬의 크기 확인

(3, 94)

In [4]:
from sklearn.decomposition import TruncatedSVD
# n_components 토픽수
svd_model = TruncatedSVD(n_components=5)
svd_model.fit(X)
len(svd_model.components_)

3

In [5]:
import numpy as np
# 토픽수 x 단어수
np.shape(svd_model.components_)

(3, 94)

In [6]:
svd_model.components_

array([[ 0.07006013,  0.11077931,  0.10479321,  0.06773035,  0.18681196,
         0.07006013,  0.07006013,  0.06773035,  0.10479321,  0.06773035,
         0.13753305,  0.06773035,  0.07006013,  0.06773035,  0.06773035,
         0.07006013,  0.07006013,  0.14680932,  0.06773035,  0.07006013,
         0.07006013,  0.06773035,  0.07006013,  0.06773035,  0.06773035,
         0.06773035,  0.25361601,  0.13753305,  0.06773035,  0.1357612 ,
         0.06773035,  0.10479321,  0.06773035,  0.11077931,  0.07006013,
         0.06773035,  0.07006013,  0.11077931,  0.11077931,  0.11077931,
         0.07006013,  0.07006013,  0.06773035,  0.06773035,  0.10479321,
         0.11077931,  0.07006013,  0.06773035,  0.11077931,  0.11077931,
         0.11077931,  0.07006013,  0.10479321,  0.06773035,  0.1354607 ,
         0.07006013,  0.06773035,  0.06773035,  0.07006013,  0.07006013,
         0.06773035,  0.06773035,  0.07006013,  0.07006013,  0.06773035,
         0.15630389,  0.11077931,  0.11077931,  0.1

In [7]:
# 단어 집합
terms = vectorizer.get_feature_names()
def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1),
              [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(svd_model.components_, terms)
#각 토픽의 핵심 키워드 추출

Topic 1: [('츄파춥스', 0.33225), ('브랜드', 0.25362), ('핸드허그', 0.22957), ('계약', 0.18681), ('제품', 0.1563)]
Topic 2: [('소식', 0.19768), ('중소기업', 0.19768), ('주년', 0.19768), ('스페인', 0.19768), ('종류', 0.19768)]
Topic 3: [('이달', 0.24144), ('츄파춥스', 0.15948), ('획득', 0.12072), ('다양', 0.12072), ('저명성', 0.12072)]


c:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
